## Adding a subset of COCO people data

In [1]:
import glob, json, os
import numpy as np
import cv2
import pycocotools.mask as mask_util
from PIL import Image, ImageDraw

from luxonis_ml.ops import *

### Download and extract data

In [2]:
! pip install gdown
! gdown 1XlvFK7aRmt8op6-hHkWVKIJQeDtOwoRT -O ../data/COCO_people_subset.zip
! unzip ../data/COCO_people_subset.zip -d ../data/

You should consider upgrading via the '/home/conor/Luxonis/envs/fiftyone/bin/python3.9 -m pip install --upgrade pip' command.
Downloading...
From: https://drive.google.com/uc?id=1XlvFK7aRmt8op6-hHkWVKIJQeDtOwoRT
To: /home/conor/Luxonis/tmp/luxonis-ml/data/COCO_people_subset.zip
100%|██████████████████████████████████████| 7.78M/7.78M [00:00<00:00, 29.9MB/s]
Archive:  ../data/COCO_people_subset.zip
  inflating: ../data/person_keypoints_val2017.json  
   creating: ../data/person_val2017_subset/
  inflating: ../data/person_val2017_subset/000000001490.jpg  
  inflating: ../data/person_val2017_subset/000000003934.jpg  
  inflating: ../data/person_val2017_subset/000000005060.jpg  
  inflating: ../data/person_val2017_subset/000000003255.jpg  
  inflating: ../data/person_val2017_subset/000000001761.jpg  
  inflating: ../data/person_val2017_subset/000000001000.jpg  
  inflating: ../data/person_val2017_subset/000000002431.jpg  
  inflating: ../data/person_val2017_subset/000000002006.jpg  
  infl

### Find image paths and load COCO annotations

In [3]:
img_dir = '../data/person_val2017_subset'
annot_file = '../data/person_keypoints_val2017.json'

# get paths to images sorted by number
im_paths = glob.glob(img_dir+'/*.jpg')
nums = np.array([int(path.split('/')[-1].split('.')[0]) for path in im_paths])
idxs = np.argsort(nums)
im_paths = list(np.array(im_paths)[idxs])

# load 
with open(annot_file) as file:
    data = json.load(file)
imgs = data['images']
anns = data['annotations']

### Convert from COCO format

The LDF will expect the following format:
```
additions (list[dict]): {
    # path to the image on local storage
    'filepath' (str): "my_image.jpg",
    # list of boxes, which are length 6 lists of [class, x, y, width, height]
    'boxes' (list[list]): [["apple", 0.1, 0.2, 0.1, 0.2], ...],
    # list of keypoint instances, which are length 2 [class, points] and points is a list of (x,y) tuples
    'keypoints' (list[str|int, list[tuple]]): [["apple", [(0.1, 0.2), (0.3, 0.4), ...], ...],
    # an integer numpy array of HxW where the values correspond to classes
    'segmentation' (np.ndarray): np.zeros((height, width))
}
```

In [4]:
# create some artificial splits
splits = ['train' for _ in range(20)] + ['val' for _ in range(10)]

additions = [] # list of additional training examples

for i, path in enumerate(im_paths):
    # find annotations matching the COCO image
    gran = path.split('/')[-1]
    img = [img for img in imgs if img['file_name']==gran][0]
    img_id = img['id']
    img_anns = [ann for ann in anns if ann['image_id'] == img_id]
    
    # load the image
    im = cv2.imread(path)
    height, width, _ = im.shape
    
    # initialize annotations for LDF
    mask = np.zeros((height, width)) # segmentation mask is always a HxW numpy array
    boxes = [] # bounding boxes are a list of [class, x, y, width, height] of the box
    keypoints = [] # keypoints are a list of classes and (x,y) points
    
    for ann in img_anns:
        # COCO-specific conversion for segmentation
        seg = ann['segmentation']
        if isinstance(seg, list):
            for s in seg:
                poly = np.array(s).reshape(-1,2)
                poly = [(poly[i,0],poly[i,1]) for i in range(len(poly))]
                m = Image.new('L', (width, height), 0)
                ImageDraw.Draw(m).polygon(poly, outline=1, fill=1)
                m = np.array(m)
                mask[m==1] = 1
        
        # COCO-specific conversion for bounding boxes
        x, y, w, h = ann['bbox']
        boxes.append(['person', x/width, y/height, w/width, h/height])
        
        # COCO-specific conversion for keypoints
        kps = np.array(ann['keypoints']).reshape(-1, 3)
        keypoint = []
        for kp in kps:
            if kp[2] == 0:
                keypoint.append((float('nan'), float('nan')))
            else: 
                keypoint.append((kp[0]/width, kp[1]/height))  
        keypoints.append(['person', keypoint])
    
    # dictionary structure expected by LDF
    additions.append({
        'image': {
            'filepath': path,
            'segmentation': mask,
            'boxes': boxes,
            'keypoints': keypoints,
            'split': splits[i]
        }
    })

In [5]:
with LuxonisDataset('luxonis', 'coco_test', bucket_type='local') as dataset:
    
    # defines the format of the COCO data source,
    # which in this case is just a single image per training example
    dataset.create_source(
        'coco',
        custom_components=[
            LDFComponent('image', HType.IMAGE, IType.BGR)
        ]
    )
    
    dataset.set_classes(['person']) # needed for classification and detection
    dataset.set_mask_targets({1:'person'}) # needed for segmentation
    dataset.set_skeleton({
        'labels': data['categories'][0]['keypoints'],
        'edges': (np.array(data['categories'][0]['skeleton'])-1).tolist()
    }) # optional for keypoints to define the skeleton visualization
    
    dataset.add(additions, note="adding initial data :)")

/home/conor/Luxonis/tmp/luxonis-ml/src/luxonis_ml/ops/dataset.py:266: UserWarning: Updating from a previously saved source!
  warnings.warn(f"Updating from a previously saved source!")


Checking for additions or modifications...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 38.69it/s]

No new additions!
